In [1]:
%load_ext autoreload
%autoreload 2

# ImageNet

## VGG-16

In [2]:
from torchvision.datasets import ImageNet
from torchvision import transforms

train_transforms = transforms.Compose([transforms.ToTensor(),
                                transforms.RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0)),
                                transforms.RandomHorizontalFlip()])
test_trainsorms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224))])

train_dataset = ImageNet(root='../dataset/cifar10',
                         split='train',)
test_dataset = ImageNet(root='../dataset/cifar10',
                       split='val')

In [3]:
from torch.utils.data import DataLoader

batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
from model.vgg import VGG16, VGG19
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
from torch.nn import CrossEntropyLoss
from evaluation.metrics import Metric
from evaluation.eval import ClassificationEvaluator

model = VGG16(10)
optimizer = Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9, weight_decay=5e-4)
scheduler = ExponentialLR(optimizer, gamma=0.9)
loss_fn = CrossEntropyLoss()
metrics = [Metric.Accuracy, ]
evaluator = ClassificationEvaluator(metrics)

In [5]:
from trainer.supervised import SimpleTrainer

trainer = SimpleTrainer(model, train_loader, test_loader, loss_fn=loss_fn, optimizer=optimizer, evaluator=evaluator, eval_freq=300)

In [6]:
epochs = 10

for epoch in range(epochs):
    trainer.train()
    scheduler.step()

Loss for id 0: 0.07207091152667999
Metrics for id 0:
Accuracy
0.1     
Loss for id 0: 0.07195968925952911
Loss for id 10: 0.0719691663980484
Loss for id 20: 0.07195316255092621
Loss for id 30: 0.0719582810997963
Loss for id 40: 0.07194732129573822
Loss for id 50: 0.07195573300123215
Loss for id 60: 0.07195300608873367
Loss for id 70: 0.07195214182138443
Loss for id 80: 0.07195375859737396
Loss for id 90: 0.0719320997595787
Loss for id 100: 0.07194479554891586
Loss for id 110: 0.07196011394262314
Loss for id 120: 0.07195963710546494
Loss for id 130: 0.07196369767189026
Loss for id 140: 0.07196101546287537
Loss for id 150: 0.07194046676158905
Loss for id 160: 0.07197922468185425
Loss for id 170: 0.07192674279212952
Loss for id 180: 0.07194458693265915
Loss for id 190: 0.0719650536775589
Loss for id 200: 0.0719548836350441
Loss for id 210: 0.07195459306240082
Loss for id 220: 0.07193824648857117
Loss for id 230: 0.07196042686700821
Loss for id 240: 0.071944959461689
Loss for id 250: 0.071